In [68]:
import pandas as pd
import numpy as np
import os
import json

# 读取数据
root_path = "/workspace/med_proj"
file_path = f"{root_path}/data/all_data.xlsx"
data = pd.read_excel(file_path)

id2cluster = pd.read_csv(f"{root_path}/data/result.csv", index_col=None, header=0)
with open(f"{root_path}/results/dataprocess/72h/name_id_mapping.json", "r", encoding="utf-8") as f:
    name2id = json.load(f)
id2name = {v: k for k, v in name2id.items()}

name2cluster = {}
for index, row in id2cluster.iterrows():
    name2cluster[id2name[row['ID']]] = row['cluster']

# df_filtered = data[data['姓名'].isin(name2id.keys())]

# # 将 name2id 中的对应ID添加到新列中
# df_filtered['cluster'] = df_filtered['姓名'].map(name2cluster)
# df_filtered.rename(columns={'姓名': 'name'}, inplace=True)
# df_filtered['性别'] = df_filtered['性别'].replace({'男': 0, '女': 1})
# df_filtered['转归'] = df_filtered['转归'].replace({'存活': 1,'死亡': 0, '好转': 1, "未愈": 0, "其他":1})
# df_filtered['机械通气'] = df_filtered['机械通气'].replace({'无': 0, '有': 1})
# df_filtered['是否二感'] = df_filtered['是否二感'].replace({'否': 0, '是': 1, '疑似': 0})
# df_filtered['慢性病个数'] = df_filtered['慢性病个数'].replace({0: 0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 2})


df_filtered = data[data['姓名'].isin(name2id.keys())].copy()
df_filtered.loc[:, 'cluster'] = df_filtered['姓名'].map(name2cluster)

df_filtered = df_filtered.rename(columns={'姓名': 'name'})

df_filtered.loc[:, '性别'] = df_filtered['性别'].map({'男': 0, '女': 1})
df_filtered['性别'] = df_filtered['性别'].astype(int)
df_filtered.loc[:, '转归'] = df_filtered['转归'].map({'存活': 1, '死亡': 0, '好转': 1, "未愈": 0, "其他": 1})
df_filtered['转归'] = df_filtered['转归'].astype(int)
df_filtered.loc[:, '机械通气'] = df_filtered['机械通气'].map({'无': 0, '有': 1})
df_filtered['机械通气'] = df_filtered['机械通气'].astype(int)
df_filtered.loc[:, '是否二感'] = df_filtered['是否二感'].map({'否': 0, '是': 1, '疑似': 0})
df_filtered['是否二感'] = df_filtered['是否二感'].astype(int)
df_filtered.loc[:, '慢性病个数'] = df_filtered['慢性病个数'].map({0: 0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 2})
df_filtered['慢性病个数'] = df_filtered['慢性病个数'].astype(int)

df_filtered.to_csv(f'{root_path}/results/multireg/cluster_feature_adjust.csv', index=False)

In [72]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

df = df_filtered

# 生成 cluster 的虚拟变量，基准类别为 cluster=4
df = pd.get_dummies(df, columns=['cluster'], drop_first=False)
df = df.rename(columns={'cluster_1': 'cluster_1', 'cluster_2': 'cluster_2', 'cluster_3': 'cluster_3'})
# df = df.drop(columns=['cluster_4'])

# 生成慢性病个数的虚拟变量，基准类别为 0 (没有慢性病)
df = pd.get_dummies(df, columns=['慢性病个数'], drop_first=True)
df = df.rename(columns={'慢性病个数_1': '慢性病个数_1', '慢性病个数_2': '慢性病个数_2'})

In [73]:
df['cluster_1'] = df['cluster_1'].astype(int)
df['cluster_2'] = df['cluster_2'].astype(int)
df['cluster_3'] = df['cluster_3'].astype(int)
df['慢性病个数_1'] = df['慢性病个数_1'].astype(int)
df['慢性病个数_2'] = df['慢性病个数_2'].astype(int)

In [74]:
# 定义自变量和因变量
X = df[['性别', '年龄', 'APACHE', 'cluster_1', 'cluster_2', 'cluster_3', '慢性病个数_1', '慢性病个数_2']]
y = df['转归']

# 添加常数项
X = sm.add_constant(X)

# 建立逻辑回归模型
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# 输出结果
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.586460
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                     转归   No. Observations:                  288
Model:                          Logit   Df Residuals:                      279
Method:                           MLE   Df Model:                            8
Date:                Tue, 14 May 2024   Pseudo R-squ.:                 0.05575
Time:                        14:42:19   Log-Likelihood:                -168.90
converged:                       True   LL-Null:                       -178.87
Covariance Type:            nonrobust   LLR p-value:                   0.01055
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2401      0.847      1.464      0.143      -0.420       2.900
性别             0.2911      0.